# EP1
## Classificação da clareza das respostas na plataforma eSIC

ACH2118 - Introdução ao Processamento de Língua Natural

Professor Ivandré Paraboni

Integrantes:
* Luiza Borghi de Mello - 11796037
* Raphael Nobuaki Iwamoto - 11882986


### Base

In [1]:
%pip install pandas
%pip install numpy
%pip install spacy
%pip install nltk
%pip install scikit-learn
%pip install openpyxl

!python -m spacy download pt_core_news_lg
!python3 -m spacy download pt_core_news_lg

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Raphael\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\Raphael\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\Raphael\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.



Note: you may need to restart the kernel to use updated packages.


...

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Raphael\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\Raphael\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


You should consider upgrading via the 'c:\Users\Raphael\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
Python n�o encontrado; execute sem argumentos para instalar na Microsoft Store ou desabilite este atalho a partir de Configura��es > Gerenciar Aliases de Execu��o do Aplicativo.


In [2]:
import pandas as pd
import numpy as np
import spacy
import nltk
import time
import itertools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectPercentile, chi2, f_classif 

In [3]:
nlp = spacy.load('pt_core_news_lg')

def aplica_lematizacao(df: pd.DataFrame, coluna: str):
    df[coluna + '_lematizada'] = ''
    i = 0
    for text in df[coluna]:
        doc = nlp(text)
        lemma = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
        df[coluna + '_lematizada'].iloc[i] = " ".join(x for x in lemma)
        i = i + 1

In [4]:
nltk.download('rslp')

def aplica_stemming(df: pd.DataFrame, coluna: str):
    stemmer = nltk.stem.RSLPStemmer()
    df[coluna + '_stemming'] = df[coluna].apply(lambda x: " ".join(stemmer.stem(x) for x in x.split()))

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Raphael\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [5]:
nltk.download('stopwords')

def remove_stopwords(df: pd.DataFrame, coluna: str):
    stop = nltk.corpus.stopwords.words('portuguese')
    df[coluna + '_stopwords'] = df[coluna].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Raphael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Leitura e análise do dataset

In [6]:
treino_dataset = pd.read_excel('data/ep1_esic2023_clareza_TRAIN.xlsx')
treino_dataset

,resp_text,clarity
0,Prezado Sr Jose Taunai Em atenção ao seu pe...,c5
1,"""A pedido do Pró-Reitor de Graduação, informa...",c5
2,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234
3,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234
4,"""Prezado Prof. Gilberto Tadeu Reis da Silva ...",c234
...,...,...
5995,Trata-se de solicitação com base na Lei de Ac...,c1
5996,Trata-se de uma solicitação repetida. As info...,c5
5997,Unidade:,c5
5998,Vale dizer que a gestão dos Telefones de Uso ...,c234


In [7]:
treino_dataset.describe()

,resp_text,clarity
count,6000,6000
unique,5626,3
top,"Prezado(a) Senhor(a), Sua manifestação foi a...",c5
freq,41,2000


In [8]:
counts = treino_dataset['clarity'].value_counts()
counts

clarity
c5      2000
c234    2000
c1      2000
Name: count, dtype: int64

In [9]:
aplica_lematizacao(treino_dataset, 'resp_text')
aplica_stemming(treino_dataset, 'resp_text')
remove_stopwords(treino_dataset, 'resp_text')

treino_dataset

,resp_text,clarity,resp_text_lematizada,resp_text_stemming,resp_text_stopwords
0,Prezado Sr Jose Taunai Em atenção ao seu pe...,c5,Prezado Sr Jose Taunai em atenção a o seu pedi...,prez sr jos taun em atenç ao seu ped de acess ...,Prezado Sr Jose Taunai Em atenção pedido acess...
1,"""A pedido do Pró-Reitor de Graduação, informa...",c5,a pedido de o Pró-Reitor de Graduação informar...,"""a ped do pró-rei de graduação, inform que a u...","""A pedido Pró-Reitor Graduação, informamos UFS..."
2,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,Prezado o sr. o Agradecemos o contato e inform...,"""prez (a) sr. (a), agradec o contat e inform q...","""Prezado (a) Sr. (a), Agradecemos contato info..."
3,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,Prezado o sr. o Agradecemos o contato e inform...,"""prez (a) sr. (a), agradec o contat e inform q...","""Prezado (a) Sr. (a), Agradecemos contato info..."
4,"""Prezado Prof. Gilberto Tadeu Reis da Silva ...",c234,Prezado Prof Gilberto Tadeu Reis de o Silva em...,"""prez prof. gilbert tad reil da silv em atend ...","""Prezado Prof. Gilberto Tadeu Reis Silva Em at..."
...,...,...,...,...,...
5995,Trata-se de solicitação com base na Lei de Ac...,c1,tratar se de solicitação com base em o Lei de ...,trata-s de solicit com bas na lei de acess à i...,Trata-se solicitação base Lei Acesso Informaçã...
5996,Trata-se de uma solicitação repetida. As info...,c5,tratar se de um solicitação repetir o informaç...,trata-s de uma solicit repetida. as inform req...,Trata-se solicitação repetida. As informações ...
5997,Unidade:,c5,Unidade,unidade:,Unidade:
5998,Vale dizer que a gestão dos Telefones de Uso ...,c234,Vale dizer que o gestão de o Telefones de Uso ...,val diz que a gest do telefon de uso públic é ...,Vale dizer gestão Telefones Uso Público feita ...


### Grid Search

#### 1. Logistic Regression | sem pré-processamento | Count Vectorizer

In [ ]:
X = treino_dataset.resp_text
Y = treino_dataset.clarity

lowercase_list = [True, False]
analyzer_list = ['word', 'char']
ngram_range_list = [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]
C_list = [0.001, 0.01, 0.1, 0.5, 1.0, 5.0, 10.0]
class_weight_list = ['balanced', None]
penalty_list = ['l1', 'l2']
solver_list = ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']

best_accuracy = 0
best_hyperparameters = {}

for lowercase, analyzer, ngram_range, C, class_weight, penalty, solver in itertools.product(lowercase_list, analyzer_list, ngram_range_list, C_list, class_weight_list, penalty_list, solver_list):
    vectorizer = CountVectorizer(analyzer=analyzer, lowercase=lowercase, ngram_range=ngram_range)
    X_count = vectorizer.fit_transform(X)

    clf = LogisticRegression(max_iter=9999, C=C, class_weight=class_weight, penalty=penalty, solver=solver, random_state=100)

    accuracy = 0
    try:
        accuracy = cross_val_score(clf, X_count, Y, scoring='accuracy', cv=10, error_score='raise').mean()

        print(f'Hyperparameters: lowercase={lowercase}, analyzer={analyzer}, ngram_range={ngram_range}, C={C}, class_weight={class_weight}, penalty={penalty}, solver={solver}')
        print(f'Mean Accuracy: {accuracy}\n')

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_hyperparameters = {
                'lowercase': lowercase,
                'analyzer': analyzer,
                'ngram_range': ngram_range,
                'C': C,
                'class_weight': class_weight,
                'penalty': penalty,
                'solver': solver
            }
    except:
        print('Combination skipped\n')

print('Best Hyperparameters:', best_hyperparameters)
print('Best Mean Accuracy:', best_accuracy)

#### 2. Logistic Regression | Lematização | TF-IDF Vectorizer

In [ ]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

lowercase_list = [True, False]
analyzer_list = ['word', 'char']
ngram_range_list = [(2, 2), (3, 3), (4, 4), (5, 5)]
C_list = np.geomspace(3.0, 5.0, num=20)
class_weight_list = ['balanced', None]
penalty_list = ['l1', 'l2']
solver_list = ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']

best_accuracy = 0
best_hyperparameters = {}

for lowercase, analyzer, ngram_range, C, class_weight, penalty, solver in itertools.product(lowercase_list, analyzer_list, ngram_range_list, C_list, class_weight_list, penalty_list, solver_list):
    vectorizer = TfidfVectorizer(analyzer=analyzer, lowercase=lowercase, ngram_range=ngram_range)
    X_tfidf = vectorizer.fit_transform(X)

    clf = LogisticRegression(max_iter=9999, C=C, class_weight=class_weight, penalty=penalty, solver=solver, random_state=100)

    accuracy = 0
    try:
        accuracy = cross_val_score(clf, X_tfidf, Y, scoring='accuracy', cv=10, error_score='raise').mean()

        print(f'Hyperparameters: lowercase={lowercase}, analyzer={analyzer}, ngram_range={ngram_range}, C={C}, class_weight={class_weight}, penalty={penalty}, solver={solver}')
        print(f'Mean Accuracy: {accuracy}\n')

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_hyperparameters = {
                'lowercase': lowercase,
                'analyzer': analyzer,
                'ngram_range': ngram_range,
                'C': C,
                'class_weight': class_weight,
                'penalty': penalty,
                'solver': solver
            }
    except:
        print('Combination skipped\n')

print('Best Hyperparameters:', best_hyperparameters)
print('Best Mean Accuracy:', best_accuracy)

#### 3. Naive Bayes | sem pré-processamento | Count Vectorizer

In [ ]:
X = treino_dataset.resp_text
Y = treino_dataset.clarity

analyzer_list = ['word', 'char']
ngram_range_list = [(1, 1), (2, 2), (3, 3), (5, 5)]
alpha_list = np.geomspace(1e-3, 1, num=50)

best_accuracy = 0
best_hyperparameters = {}

for analyzer, ngram_range, alpha in itertools.product(analyzer_list, ngram_range_list, alpha_list):
    vectorizer = CountVectorizer(ngram_range=ngram_range, analyzer=analyzer)
    X_count = vectorizer.fit_transform(X)

    clf = MultinomialNB(alpha=alpha)

    accuracy = cross_val_score(clf, X_count, Y, scoring='accuracy', cv=10).mean()

    print(f'Hyperparameters: ngram_range={ngram_range}, analyzer={analyzer}, alpha={alpha}')
    print(f'Mean Accuracy: {accuracy}\n')

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = {
            'ngram_range': ngram_range,
            'analyzer': analyzer,
            'alpha': alpha
        }

print('Best Hyperparameters:', best_hyperparameters)
print('Best Mean Accuracy:', best_accuracy)

#### 4. Naive Bayes | Lematização | TF-IDF Vectorizer

In [ ]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

analyzer_list = ['word', 'char']
ngram_range_list = [(1, 1), (2, 2), (3, 3), (5, 5)]
alpha_list = np.geomspace(1e-3, 1000, num=100)

best_accuracy = 0
best_hyperparameters = {}

for analyzer, ngram_range, alpha in itertools.product(analyzer_list, ngram_range_list, alpha_list):
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, analyzer=analyzer)
    X_tfidf = vectorizer.fit_transform(X)

    clf = MultinomialNB(alpha=alpha)

    accuracy = cross_val_score(clf, X_tfidf, Y, scoring='accuracy', cv=10).mean()

    print(f'Hyperparameters: ngram_range={ngram_range}, analyzer={analyzer}, alpha={alpha}')
    print(f'Mean Accuracy: {accuracy}\n')

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = {
            'ngram_range': ngram_range,
            'analyzer': analyzer,
            'alpha': alpha
        }

print('Best Hyperparameters:', best_hyperparameters)
print('Best Mean Accuracy:', best_accuracy)

#### 5. Random Forest | Lematização | Count Vectorizer

In [ ]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

analyzer = 'word'
ngram_range = (1, 1)

vectorizer = CountVectorizer(ngram_range=ngram_range, analyzer=analyzer)
X_count = vectorizer.fit_transform(X)

clf = RandomForestClassifier(random_state=100)

accuracy = cross_val_score(clf, X_count, Y, scoring='accuracy', cv=10).mean()

print(f'Hyperparameters: ngram_range={ngram_range}, analyzer={analyzer}')
print(f'Mean Accuracy: {accuracy}\n')

#### 6. Random Forest | Lematização | TF-IDF Vectorizer

In [ ]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

analyzer_list = ['word', 'char']
ngram_range_list = [(1, 1), (2, 2)]

best_accuracy = 0
best_hyperparameters = {}

for analyzer, ngram_range in itertools.product(analyzer_list, ngram_range_list):
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, analyzer=analyzer)
    X_tfidf = vectorizer.fit_transform(X)

    clf = RandomForestClassifier(random_state=100)

    accuracy = cross_val_score(clf, X_tfidf, Y, scoring='accuracy', cv=10).mean()

    print(f'Hyperparameters: ngram_range={ngram_range}, analyzer={analyzer}')
    print(f'Mean Accuracy: {accuracy}\n')

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = {
            'ngram_range': ngram_range,
            'analyzer': analyzer,
        }

print('Best Hyperparameters:', best_hyperparameters)
print('Best Mean Accuracy:', best_accuracy)

#### 7. SVC | Lematização | Count Vectorizer

In [ ]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

analyzer = 'word'
ngram_range = (1, 1)

vectorizer = CountVectorizer(ngram_range=ngram_range, analyzer=analyzer)
X_count = vectorizer.fit_transform(X)

clf = SVC(random_state=100)

accuracy = cross_val_score(clf, X_count, Y, scoring='accuracy', cv=10).mean()

print(f'Hyperparameters: ngram_range={ngram_range}, analyzer={analyzer}')
print(f'Mean Accuracy: {accuracy}\n')

#### 8. SVC | Lematização | TF-IDF Vectorizer

In [ ]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

analyzer = 'char'
ngram_range = (3, 3)
C = 1.0

vectorizer = TfidfVectorizer(ngram_range=ngram_range, analyzer=analyzer)
X_tfidf = vectorizer.fit_transform(X)

clf = SVC(C=C, random_state=100)

accuracy = cross_val_score(clf, X_tfidf, Y, scoring='accuracy', cv=10).mean()

print(f'Hyperparameters: ngram_range={ngram_range}, analyzer={analyzer}')
print(f'Mean Accuracy: {accuracy}\n')

#### 9. SGD Classifier | Lematização | TF-IDF Vectorizer

In [ ]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

alpha_list = np.geomspace(6.632895500464648e-05, 0.00014590812272681345, num=30)
max_iter_list = [100, 1000]

best_accuracy = 0
best_hyperparameters = {}

for alpha, max_iter in itertools.product(alpha_list, max_iter_list):
    vectorizer = TfidfVectorizer(ngram_range=(3, 3), analyzer='char', lowercase=True)
    X_tfidf = vectorizer.fit_transform(X)

    clf = SGDClassifier(loss='log_loss', penalty='l2', alpha=alpha, random_state=100, max_iter=max_iter, tol=None)
    try:
        accuracy = cross_val_score(clf, X_tfidf, Y, scoring='accuracy', cv=10, n_jobs=4).mean()

        print(f'Hyperparameters: alpha={alpha}, max_iter={max_iter}')
        print(f'Mean Accuracy: {accuracy}\n')

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_hyperparameters = {
                'alpha': alpha,
                'max_iter': max_iter
            }
    except:
        print('Combination skipped\n')

print('Best Hyperparameters:', best_hyperparameters)
print('Best Mean Accuracy:', best_accuracy)

#### 10. SGD Classifier | Lematização | SelectPercentile | TF-IDF Vectorizer 

In [ ]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

alpha_list = np.geomspace(6.632895500464648e-05, 0.00014590812272681345, num=30)
max_iter_list = [100, 1000]
percentile_list = [50, 75]
score_func_list = [chi2, f_classif]

best_accuracy = 0
best_hyperparameters = {}

for alpha, max_iter, percentile, score_func in itertools.product(alpha_list, max_iter_list, percentile_list, score_func_list):
  vectorizer = TfidfVectorizer(max_features=None, ngram_range=(3, 3), analyzer='char', lowercase=True)
  X_tfidf = vectorizer.fit_transform(X)
  X_BestFeatures = SelectPercentile(score_func=score_func, percentile=percentile).fit_transform(X_tfidf, Y)

  clf = SGDClassifier(loss='log_loss', penalty='l2', alpha=alpha, random_state=100, max_iter=max_iter, tol=None)
  try:
    accuracy = cross_val_score(clf, X_BestFeatures, Y, scoring='accuracy', cv=10, n_jobs=4).mean()

    print(f'Hyperparameters: alpha={alpha}, max_iter={max_iter}, percentile={percentile}, score_func={score_func}')
    print(f'Mean Accuracy: {accuracy}\n')

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = { 
            'alpha': alpha,
            'max_iter': max_iter,
            'percentile': percentile,
            'score_func': score_func
        }
  except:
      print('Combination skipped\n')  

print('Best Hyperparameters:', best_hyperparameters)
print('Best Mean Accuracy:', best_accuracy)

#### 11. SGD Classifier | Lematização | SelectPercentile | TF-IDF Vectorizer

In [ ]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

alpha_list = np.geomspace(6.632895500464648e-05, 0.00014590812272681345, num=20)
max_iter_list = [100, 1000]
percentile_list = [50, 75]
score_func_list = [chi2, f_classif]
analyzer_list = ['word', 'char']
ngram_range_list = [(1, 1), (2, 2), (3, 3), (5, 5)]

best_accuracy = 0
best_hyperparameters = {}

for alpha, max_iter, percentile, score_func, analyzer, ngram_range in itertools.product(alpha_list, max_iter_list, percentile_list, score_func_list, analyzer_list, ngram_range_list):
  vectorizer = TfidfVectorizer(max_features=None, ngram_range= ngram_range, analyzer= analyzer, lowercase=True)
  X_tfidf = vectorizer.fit_transform(X)
  X_BestFeatures = SelectPercentile(score_func=score_func , percentile=percentile).fit_transform(X_tfidf, Y)

  clf = SGDClassifier(loss='log_loss', penalty='l2', alpha=alpha, random_state=100, max_iter=max_iter, tol=None)
  try:
    accuracy = cross_val_score(clf, X_BestFeatures, Y, scoring='accuracy', cv=10, n_jobs=4).mean()

    print(f'Hyperparameters: alpha={alpha}, max_iter={max_iter}, percentile={percentile}, score_func={score_func}, analyzer={analyzer}, ngram_range={ngram_range}')
    print(f'Mean Accuracy: {accuracy}\n')

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = {
            'alpha': alpha,
            'max_iter': max_iter,
            'percentile': percentile,
            'score_func': score_func,
            'analyzer': analyzer,
            'ngram_range': ngram_range
        }
  except:
      print('Combination skipped\n')  

print('Best Hyperparameters:', best_hyperparameters)
print('Best Mean Accuracy:', best_accuracy)

#### 12. SGD Classifier otimização de kBest e alpha

In [10]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

alpha_list = np.geomspace(7e-07, 7e-03, num=20)
percentile_list = list(range(10,70,5))

best_accuracy = 0
best_hyperparameters = {}

vectorizer = TfidfVectorizer(max_features=None, ngram_range= (2,2), analyzer= 'word', lowercase=True)
X_tfidf = vectorizer.fit_transform(X)

for percentile, alpha in itertools.product(percentile_list, alpha_list):
  clf = SGDClassifier(loss='log_loss', penalty='l2', alpha=alpha, random_state=100, max_iter=100, tol=None)
  X_BestFeatures = SelectPercentile(score_func=f_classif , percentile=percentile).fit_transform(X_tfidf, Y)
  try:
    result = cross_val_score(clf, X_BestFeatures, Y, scoring='accuracy', cv=10, n_jobs=4)
    accuracy = result.mean()
    std_deviation = result.std()

    print(f'Hyperparameters: alpha={alpha}, max_iter= 100, percentile={percentile}, score_func= f_classif, analyzer=word, ngram_range=(2,2), shape={X_BestFeatures.shape}, std_deviation={std_deviation}')
    print(f'Mean Accuracy: {accuracy}\n')

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = {
            'alpha': alpha,
            'max_iter': 100,
            'percentile': percentile,
            'score_func': 'f_classif',
            'analyzer': 'word',
            'ngram_range': (2,2),
            'std_deviation': std_deviation,
            'X_Best_shape': X_BestFeatures.shape
        }
  except:
      print('Combination skipped\n')  

print('Best Hyperparameters:', best_hyperparameters)
print('Best Mean Accuracy:', best_accuracy)

Hyperparameters: alpha=7e-07, max_iter= 100, percentile=10, score_func= f_classif, analyzer=word, ngram_range=(2,2), shape=(6000, 17658), std_deviation=0.037686204372422524
Mean Accuracy: 0.5698333333333332

Hyperparameters: alpha=1.1366437174321049e-06, max_iter= 100, percentile=10, score_func= f_classif, analyzer=word, ngram_range=(2,2), shape=(6000, 17658), std_deviation=0.035660825066793474
Mean Accuracy: 0.5711666666666667

Hyperparameters: alpha=1.8456556291112491e-06, max_iter= 100, percentile=10, score_func= f_classif, analyzer=word, ngram_range=(2,2), shape=(6000, 17658), std_deviation=0.035744463440743744
Mean Accuracy: 0.5730000000000001

Hyperparameters: alpha=2.996932679103578e-06, max_iter= 100, percentile=10, score_func= f_classif, analyzer=word, ngram_range=(2,2), shape=(6000, 17658), std_deviation=0.036507609556852054
Mean Accuracy: 0.5711666666666668

Hyperparameters: alpha=4.8663495732429244e-06, max_iter= 100, percentile=10, score_func= f_classif, analyzer=word, ngr

#### 13. SGD Classifier otimização de alpha

##### Visualização das features

In [34]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

alpha_list = np.geomspace(7e-10, 7e-03, num=200)

best_accuracy = 0
best_hyperparameters = {}

vectorizer = TfidfVectorizer(max_features=None, ngram_range= (2,2), analyzer= 'word', lowercase=True)
X_tfidf = vectorizer.fit_transform(X)
selector = SelectPercentile(score_func=f_classif , percentile=15)
X_BestFeatures = selector.fit_transform(X_tfidf, Y)

for name in selector.get_feature_names_out():
    print(name)

x7
x8
x29
x30
x31
x33
x36
x40
x41
x55
x62
x71
x77
x100
x110
x115
x118
x147
x162
x182
x188
x193
x207
x208
x212
x275
x313
x329
x335
x391
x505
x506
x515
x551
x633
x682
x697
x755
x785
x796
x797
x818
x861
x882
x932
x977
x993
x994
x1002
x1058
x1061
x1067
x1068
x1070
x1071
x1082
x1084
x1086
x1089
x1109
x1124
x1129
x1135
x1155
x1171
x1172
x1173
x1181
x1183
x1193
x1201
x1202
x1206
x1207
x1214
x1217
x1224
x1244
x1245
x1367
x1384
x1393
x1404
x1405
x1408
x1417
x1445
x1447
x1496
x1507
x1538
x1563
x1576
x1587
x1589
x1592
x1595
x1596
x1597
x1607
x1608
x1609
x1611
x1612
x1626
x1639
x1657
x1658
x1669
x1670
x1682
x1689
x1715
x1736
x1741
x1750
x1766
x1770
x1773
x1774
x1786
x1791
x1792
x1805
x1812
x1815
x1824
x1859
x1864
x1879
x1890
x1893
x1897
x1909
x1912
x1913
x1915
x1916
x1917
x1918
x1919
x1920
x1930
x1936
x1954
x1955
x1957
x1962
x2001
x2002
x2003
x2005
x2007
x2011
x2019
x2023
x2024
x2025
x2027
x2028
x2057
x2066
x2067
x2068
x2088
x2089
x2090
x2102
x2107
x2112
x2114
x2115
x2116
x2117
x2118
x2132
x2136
x

##### Otimização com 15% e bigramas

In [15]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

alpha_list = np.geomspace(7e-10, 7e-03, num=200)

best_accuracy = 0
best_hyperparameters = {}

vectorizer = TfidfVectorizer(max_features=None, ngram_range= (2,2), analyzer= 'word', lowercase=True)
X_tfidf = vectorizer.fit_transform(X)
X_BestFeatures = SelectPercentile(score_func=f_classif , percentile=15).fit_transform(X_tfidf, Y)

for alpha in alpha_list:
  clf = SGDClassifier(loss='log_loss', penalty='l2', alpha=alpha, random_state=100, max_iter=100, tol=None)
  try:
    result = cross_val_score(clf, X_BestFeatures, Y, scoring='accuracy', cv=10, n_jobs=4)
    accuracy = result.mean()
    std_deviation = result.std()
    min_accuracy = result.min()
    max_accuracy = result.max()

    print(f'Hyperparameters: alpha={alpha}, \nmax_iter= 100, \npercentile=15, \nscore_func= f_classif,\nanalyzer=word,\nngram_range=(2,2),\nshape={X_BestFeatures.shape}, \nstd_deviation={std_deviation}, \nmin={min_accuracy}, \nmax={max_accuracy}')
    print(f'Mean Accuracy: {accuracy}\n\n')

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = {
            'alpha': alpha,
            'max_iter': 100,
            'percentile': 15,
            'score_func': 'f_classif',
            'analyzer': 'word',
            'ngram_range': (2,2),
            'std_deviation': std_deviation,
            'min': min_accuracy,
            'max': max_accuracy
        }
  except:
      print('Combination skipped\n')  

print('Best Hyperparameters:', best_hyperparameters)
print('Best Mean Accuracy:', best_accuracy)

Hyperparameters: alpha=7e-10, 
max_iter= 100, 
percentile=15, 
score_func= f_classif,
analyzer=word,
ngram_range=(2,2),
shape=(6000, 26487), 
std_deviation=0.04043135471938141, 
min=0.485, 
max=0.6366666666666667
Mean Accuracy: 0.5498333333333333


Hyperparameters: alpha=7.590561780827262e-10, 
max_iter= 100, 
percentile=15, 
score_func= f_classif,
analyzer=word,
ngram_range=(2,2),
shape=(6000, 26487), 
std_deviation=0.045021291259432654, 
min=0.4666666666666667, 
max=0.6416666666666667
Mean Accuracy: 0.5501666666666667


Hyperparameters: alpha=8.230946878365109e-10, 
max_iter= 100, 
percentile=15, 
score_func= f_classif,
analyzer=word,
ngram_range=(2,2),
shape=(6000, 26487), 
std_deviation=0.0393403948463493, 
min=0.4766666666666667, 
max=0.635
Mean Accuracy: 0.5463333333333333


Hyperparameters: alpha=8.925358684991092e-10, 
max_iter= 100, 
percentile=15, 
score_func= f_classif,
analyzer=word,
ngram_range=(2,2),
shape=(6000, 26487), 
std_deviation=0.04454835076134195, 
min=0.47833333

##### Otimização uma e duas palavras

In [17]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

alpha_list = np.geomspace(7e-7, 7e-5, num=20)
percentile_list = list(range(10,60,5))

best_accuracy = 0
best_hyperparameters = {}

vectorizer = TfidfVectorizer(max_features=None, ngram_range= (1,2), analyzer= 'word', lowercase=True)
X_tfidf = vectorizer.fit_transform(X)


for percentile, alpha in itertools.product(percentile_list, alpha_list):
  X_BestFeatures = SelectPercentile(score_func=f_classif , percentile=percentile).fit_transform(X_tfidf, Y)
  clf = SGDClassifier(loss='log_loss', penalty='l2', alpha=alpha, random_state=100, max_iter=100, tol=None)
  try:
    result = cross_val_score(clf, X_BestFeatures, Y, scoring='accuracy', cv=10, n_jobs=4)
    accuracy = result.mean()
    std_deviation = result.std()
    min_accuracy = result.min()
    max_accuracy = result.max()

    print(f'Hyperparameters: alpha={alpha}, \nmax_iter= 100, \npercentile={percentile}, \nscore_func= f_classif,\nanalyzer=word,\nngram_range=(1,2),\nshape={X_BestFeatures.shape}, \nstd_deviation={std_deviation}, \nmin={min_accuracy}, \nmax={max_accuracy}')
    print(f'Mean Accuracy: {accuracy}\n\n')

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = {
            'alpha': alpha,
            'max_iter': 100,
            'percentile': percentile,
            'score_func': 'f_classif',
            'analyzer': 'word',
            'ngram_range': (1,2),
            'std_deviation': std_deviation,
            'min': min_accuracy,
            'max': max_accuracy
        }
  except:
      print('Combination skipped\n')  

print('Best Hyperparameters:', best_hyperparameters)
print('Best Mean Accuracy:', best_accuracy)

Hyperparameters: alpha=7e-07, 
max_iter= 100, 
percentile=10, 
score_func= f_classif,
analyzer=word,
ngram_range=(1,2),
shape=(6000, 20136), 
std_deviation=0.039101008796307145, 
min=0.49333333333333335, 
max=0.635
Mean Accuracy: 0.5616666666666668


Hyperparameters: alpha=8.919924899921946e-07, 
max_iter= 100, 
percentile=10, 
score_func= f_classif,
analyzer=word,
ngram_range=(1,2),
shape=(6000, 20136), 
std_deviation=0.039145384288714176, 
min=0.495, 
max=0.6383333333333333
Mean Accuracy: 0.5625


Hyperparameters: alpha=1.1366437174321049e-06, 
max_iter= 100, 
percentile=10, 
score_func= f_classif,
analyzer=word,
ngram_range=(1,2),
shape=(6000, 20136), 
std_deviation=0.039973602400695515, 
min=0.49, 
max=0.6366666666666667
Mean Accuracy: 0.5623333333333334


Hyperparameters: alpha=1.4483966567803534e-06, 
max_iter= 100, 
percentile=10, 
score_func= f_classif,
analyzer=word,
ngram_range=(1,2),
shape=(6000, 20136), 
std_deviation=0.041097783124856985, 
min=0.48333333333333334, 
max=0.6

### Predict

Classificador: SGDClassifier

Pré-processamento: Lematização

Representação: TF-IDF

Parâmetros:

alpha: 1.8456556291112491e-06

loss: log_loss

penalty: l2

max_iter: 100

percentile: 15

score_func: f_classif

analyzer: word

ngram_range: (1, 2)

In [50]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

alpha_list = np.geomspace(7e-10, 7e-03, num=150)

best_accuracy = 0
best_hyperparameters = {}

vectorizer = TfidfVectorizer(max_features=None, ngram_range= (1,2), analyzer= 'word', lowercase=True)
X_tfidf = vectorizer.fit_transform(X)
selector = SelectPercentile(score_func=f_classif , percentile=15)
X_BestFeatures = selector.fit_transform(X_tfidf, Y)

clf = SGDClassifier(loss='log_loss', penalty='l2', alpha=1.8456556291112491e-06, random_state=100, max_iter=100, tol=None)

clf.fit(X_BestFeatures, Y)

X_test = selector.transform(X_tfidf) #passar conjunto de testes

clf.predict(X_test)

array(['c5', 'c5', 'c234', ..., 'c5', 'c234', 'c234'], dtype='<U4')